In [ ]:
!pip install trankit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 28.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 74.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 12.0 MB/s eta 0:00:00
   

In [ ]:
import pandas as pd
import json
from trankit import Pipeline


# initialize a multilingual pipeline
p = Pipeline(lang='hindi', gpu=True, cache_dir='./cache')

# function for grouping
def group_words_by_rules(words, pos_tags):
    grouped_words = []
    grouped_pos_tags = []
    i = 0

    while i < len(words):
        word = words[i]
        pos = pos_tags[i]

        if word == 'लिए' and i>0 and words[i-1] == 'के':
            grouped_words[-1] += f"##{word}"
        elif pos == 'VERB' and i>0 and words[i-1] == 'नहीं':
            grouped_words[-1] +=  f"##{word}"
            grouped_pos_tags[-1] = pos
        else:
            grouped_words.append(word)
            grouped_pos_tags.append(pos)
        i+=1
    i=0
    words = grouped_words
    grouped_words = []
    pos_tags = grouped_pos_tags
    grouped_pos_tags = []
    while i < len(words):
        word = words[i]
        pos = pos_tags[i]

        # Rule 1: Group postpositions and particles
        if word in ['ने', 'को', 'से', 'में', 'का', 'के', 'की', 'पर', 'ही', 'भी', 'सा', 'वाला', 'के##लिए']:
            if (word != 'की' or (i!=len(words)-1 and pos_tags[i-1]=='NOUN')) and i>0:
                grouped_words[-1] += f"##{word}"
            else:
                 grouped_words.append(word)
            i += 1


        # Rule 2: Verbs ending with 'ता/ना' followed by forms of 'है/था'
        elif pos == 'VERB':
            grouped_words.append(word)
            if i < len(words)-1 and words[i+1] in ['है', 'थी','था', 'थे', 'हूँ','हैं','हो'] and word[-1] in ["ता", "ती", "ते", "ना", "ने", "नी"]:
                grouped_words[-1] += f"##{words[i+1]}"
                i+=2
            else:
                i+=1
                while i<len(words) and pos_tags[i] == 'AUX':
                    grouped_words[-1] += f"##{words[i]}"
                    i+=1

        else:
            grouped_words.append(word)
            i += 1
    return ' '.join(grouped_words)


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

http://nlp.uoregon.edu/download/trankit/v1.0.0/xlm-roberta-base/hindi.zip


Downloading: 100%|██████████| 27.1M/27.1M [00:00<00:00, 65.5MiB/s]


Loading pretrained XLM-Roberta, this may take a while...


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/adapters/composition.py:186: FutureWarning: Passing list objects for adapter activation is deprecated. Please use Stack or Fuse explicitly.
  warnings.warn(


Loading tokenizer for hindi
Loading tagger for hindi
Loading lemmatizer for hindi
Active language: hindi


In [ ]:
## code for loding hindi sentences
file_path = "./ungrouped_test.json"
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)
hindi_sent = [d['sourceText'] for d in data]
san_sent = [d['targetText'] for d in data]


In [ ]:
all_pos = [p.posdep(sentence) for sentence in hindi_sent]  # Batch POS generation

grouped_sentences = []

# Process each sentence based on the pre-generated POS tagging
for i, pos in enumerate(all_pos):
    sentence = pos['sentences'][0]
    upos = [token['upos'] for token in sentence['tokens']]
    words = [token['text'] for token in sentence['tokens']]
    grouped_sent = group_words_by_rules(words, upos)
    grouped_sentences.append(grouped_sent)

Checkpoint saved at iteration 7000.
Checkpoint saved at iteration 7100.
Checkpoint saved at iteration 7200.
Checkpoint saved at iteration 7300.
Checkpoint saved at iteration 7400.
Checkpoint saved at iteration 7500.
Checkpoint saved at iteration 7600.
Checkpoint saved at iteration 7700.
Checkpoint saved at iteration 7800.
Checkpoint saved at iteration 7900.
Checkpoint saved at iteration 8000.
Checkpoint saved at iteration 8100.
Checkpoint saved at iteration 8200.
Checkpoint saved at iteration 8300.
Checkpoint saved at iteration 8400.
Checkpoint saved at iteration 8500.
Checkpoint saved at iteration 8600.
Checkpoint saved at iteration 8700.
Checkpoint saved at iteration 8800.
Checkpoint saved at iteration 8900.
Checkpoint saved at iteration 9000.
Checkpoint saved at iteration 9100.
Checkpoint saved at iteration 9200.
Checkpoint saved at iteration 9300.
Checkpoint saved at iteration 9400.
Checkpoint saved at iteration 9500.
Checkpoint saved at iteration 9600.
Checkpoint saved at iteratio

In [ ]:
import json

# Assert that both lists have the same length
assert len(grouped_sentences) == len(san_sent), "Source and target lists must have the same number of elements."

# Convert to JSONL
jsonl_data = [{"translation":{"hin": src, "san": tgt}} for src, tgt in zip(grouped_sentences, san_sent)]
# Write to a JSONL file
output_file = "grouped_test.jsonl"
with open(output_file, "w", encoding="utf-8") as f:
    for entry in jsonl_data:
        json.dump(entry, f, ensure_ascii=False)
        f.write("\n")
